In [ ]:
%%writefile /mnt/solexa/mel_yang/BGI/freq2geno/makegenofile_rm9_ALL.py

import tabix
import numpy.random as npr
import numpy as np

pD="/mnt/solexa/mel_yang/BGI/freq2geno/"
datpD="/mnt/solexa/mel_yang/BGI/"
UPAfile="UPA_all8"

header=['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 
        'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 
        'Hubei', 'Hunan', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 
        'Neimenggu', 'Ningxia', 'Qinghai', 'Shandong', 'Shanghai', 'Shanxi1', 
        'Shanxi2', 'Sichuan', 'Tianjin', 'Xinjiang', 'Xizang', 'Yunnan', 'Zhejiang']

numchr=40
allmissing=''.join(['9' for i in header for j in range(numchr/2)])
Hantbsnps = tabix.open(datpD+"BGI/OnlyHan_siteinfo.txt.gz")
Hantbfreqs = tabix.open(datpD+"BGI/OnlyHan_freqs.txt.gz")
Protbsnps = tabix.open(datpD+"BGI/ByProvince_siteinfo.txt.gz")
Protbfreqs = tabix.open(datpD+"BGI/ByProvince_freqs.txt.gz")
Minbsnps = tabix.open(datpD+"BGI/Minority_siteinfo.txt.gz")
Minbfreqs = tabix.open(datpD+"BGI/Minority_freqs.txt.gz")

num=0
confile=open(pD+"BGI.rm9.n%s.conflict" % str(numchr/2),'w')
genofile=open(pD+"BGI.rm9.n%s.geno" % str(numchr/2),'w')
snpfilenew=open(pD+"BGI.rm9.n%s.snp" % str(numchr/2),'w')
snpfile=open(pD+UPAfile+".snp",'r')
for line in snpfile:
    #if num==5: break
    
    x=line.split()
    num+=1
    myadd="chr"+x[1]+'\t'+x[3]+'\t'
    if x[1] == '23': mychrname='X'
    elif x[1] == '24': mychrname='Y'
    else: mychrname=x[1]
    #print myadd
    snps=x[4]+x[5]
    
    Hanrecordsnps=Hantbsnps.query("chr"+mychrname,int(x[3])-1,int(x[3]))
    Hanrecordfreqs=Hantbfreqs.query("chr"+mychrname,int(x[3])-1,int(x[3]))
    Prorecordsnps=Protbsnps.query("chr"+mychrname,int(x[3])-1,int(x[3]))
    Prorecordfreqs=Protbfreqs.query("chr"+mychrname,int(x[3])-1,int(x[3]))
    Hanmysnps=[record for record in Hanrecordsnps]
    Hanmyfreqs=[record for record in Hanrecordfreqs]
    Promysnps=[record for record in Prorecordsnps]
    Promyfreqs=[record for record in Prorecordfreqs]

    if len(Hanmysnps)!=1 or len(Hanmyfreqs)!=1 or len(Promysnps)!=1 or len(Promyfreqs)!=1:
        if len(Hanmysnps)>1: print Hanmysnps
        if len(Hanmyfreqs)>1: print Hanmyfreqs
        if len(Promysnps)>1: print Promysnps
        if len(Promyfreqs)>1: print Promyfreqs
        #genofile.write(myadd+allmissing+'\n')
        continue
    if Promysnps!=Hanmysnps: print Promysnps, Hanmysnps
    #if "-nan" in myfreqs[0] or "nan" in myfreqs[0]: 
    #    #genofile.write(myadd+allmissing+'\n')
    #    confile.write('nan in %s, %s\n' % ('chr'+x[1],x[3]))
    #    ##Number here is: 202060!!
    #    continue
    myfreqs=Hanmyfreqs[0][2:]+Promyfreqs[0][2:]
    
    ##Make sure SNP alleles are correct
    mysnpstr=Hanmysnps[0][-1]
    if mysnpstr==snps: myfreqs=[float(i) if i not in ["-nan","nan"] else 4.5 for i in myfreqs]   ##ALTFREQ
    elif mysnpstr==snps[::-1]: myfreqs=[1-float(i) if i not in ["-nan","nan"] else 4.5 for i in myfreqs]   ##ALTFREQ
    else:
        #print '\t'.join(mysnps[0])+'\t'.join(x) + '\n'
        #genofile.write(myadd+allmissing+'\n')
        confile.write('\t'.join(Hanmysnps[0])+'\t'+'\t'.join(x) + '\n') ; continue

    ##Make set
    #print myfreqs, x[1],x[3]
    myary = npr.rand(numchr,len(myfreqs))
    #print myary.shape
    myfreqs1=np.array(myfreqs*numchr).reshape(numchr,len(myfreqs))
    #print np.sum(myary), np.sum(myfreqs1)
    
    myary[(myary<=myfreqs1) & (myfreqs1!=4.5)]=0  ##AMT REF, If freq=1.0, then no ref, and all unif<1, and all value 0
    myary[(myary>myfreqs1) & (myfreqs1!=4.5)]=1
    myary[(myfreqs1==4.5)]=4.5
    #print myary
    genoary=np.array([5]*(numchr/2)*len(myfreqs)).reshape(numchr/2,len(myfreqs))
    for ind in range(myary.shape[0]):
        if ind%2!=0: continue
        genoary[ind/2,]=myary[ind,]+myary[(ind+1),]
    
    #print genoary
    #if num > 600: print myary.shape, x[3]
    newary=np.transpose(genoary).reshape(1,((numchr)/2)*len(myfreqs))
    mystr=''.join([str(i) for i in list(newary[0,])])
    #if 4.5 in myfreqs:
    #    print myfreqs
    #    print myary
    #    print mystr
    genofile.write(myadd+mystr+'\n')
    snpfilenew.write(line)
    #print len(mystr)
    
    
snpfile.close()
genofile.close()
confile.close()
